In [ ]:
!pip install pykeen

In [2]:
import pykeen

In [3]:
from pykeen.datasets import PharmKG

INFO:pykeen.utils:Using opt_einsum


In [4]:
pharm_dataset = PharmKG()

In [5]:
print(pharm_dataset)

INFO:pykeen.datasets.base:downloading data from https://zenodo.org/record/4077338/files/raw_PharmKG-180k.zip to /root/.data/pykeen/datasets/pharmkg/raw_PharmKG-180k.zip


INFO:pykeen.datasets.base:reordering columns: ['Entity1_name', 'relationship_type', 'Entity2_name']
INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [691412, 109324, 109324]


PharmKG(num_entities=188296, num_relations=39, create_inverse_triples=False)


In [6]:
training_data = pharm_dataset.training.mapped_triples
print(training_data[:5])

tensor([[     0,      2, 161966],
        [     0,     21,  26679],
        [     1,     17,  95159],
        [     2,     31,  23633],
        [     3,      3,  65653]])


In [7]:
from pykeen.pipeline import pipeline

In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
pipeline_results = pipeline(
    model = "TransE",
    dataset = pharm_dataset,
    training_kwargs = dict(num_epochs=20),
    device = device
)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=128.


Evaluating on cuda:0:   0%|          | 0.00/109k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 215.11s seconds


In [10]:
print(pipeline_results)

PipelineResult(random_seed=1580601266, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(188296, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(39, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=188296, num_relations=39, create_inverse_triples=False, num_triples=874588, path="/root/.data/pykeen/datasets/pharmkg/raw_PharmKG-180k.zip"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7c6bf02ef550>, losses=[0.67690853325812, 0.25949629938476854, 0.16709667487401939, 0.131076085952759, 0.10921941388083335, 0.09560552339936544, 0.08504775604562938, 0.07801344634990683, 0.07176981989772899, 0.06556653492327126, 0.06246469597549406, 0.05881828457915123, 0.05560247996265441, 0.05372001273671472, 0.0511476033009351, 0.

In [11]:
from pykeen.predict import predict_triples

In [12]:
pack = predict_triples(model=pipeline_results.model, triples=pharm_dataset.validation)
df = pack.process(factory=pipeline_results.training).df

In [13]:
df.nlargest(n=5, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
5586,20915,ahnak,16,I,20915,ahnak,-0.827612
9185,72841,fbxl7,16,I,72841,fbxl7,-0.827612
9926,73821,fhl2,16,I,73821,fhl2,-0.827612
10531,148069,ptc,16,I,148069,ptc,-0.827612
11732,45377,cdh2,16,I,45377,cdh2,-0.827612
